<a href="https://colab.research.google.com/github/scottthomaswx/Nadocast_Zoom/blob/main/Nadocast_Zoom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Needed Packages

In [1]:
# @title
%%capture
!pip install xarray
!pip install xarray['complete']
!pip install cfgrib
!pip install cfgrib xarray
!apt-get install -y libeccodes0
!pip install eccodes
!pip install matplotlib
!pip install metpy
import xarray as xr
import requests
from metpy.plots import USCOUNTIES
import cartopy.feature as cfeat
import cartopy.crs as ccrs
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from metpy.plots import MapPanel, PanelContainer, PlotGeometry

#Set Nadocast Runtime

## Leading Zeroes Required In Day/Month

### Period 1 = Day 1 (Available For All Run)
### Period 2 = Day 2 (Available Only at 12z)


In [50]:
year = '2025'
month = '05'
day = '15'
time = '12'
period = '1'

#Product Options
## Individual Parameters: hail, wind, tornado, sig_hail, sig_wind, sig_tornado

# ...Under Development...
## Single Hazard: hail_all, wind_all, tornado_all
## All Hazard: all_hazard

In [81]:
product = 'wind' #@param ['hail', 'wind', 'tornado', 'sig_hail', 'sig_wind', 'sig_tornado']
type_spcabs = 'Absolutey Calibrated' #@param ['SPC Like', 'Absolutey Calibrated']

# Set Map Boundaries

In [71]:
WestLong = -90 # @param {type:"number"}
EastLong = -82 # @param {type:"number"}
NorthLat = 45 # @param {type:"number"}
SouthLat = 40 # @param {type:"number"}

#Pull Nadocast

In [74]:
# @title
if type_spcabs == 'SPC Like':
  if product == 'tornado' or product == 'wind' or product == 'hail':
    if period == '1':
      if time == '00' or time == '0':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_{product}_{year}{month}{day}_t{time}z_f12-35.grib2'
        sigurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_sig_{product}_{year}{month}{day}_t{time}z_f12-35.grib2'
      elif time == '12':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{product}_{year}{month}{day}_t{time}z_f02-23.grib2'
        sigurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_{product}_{year}{month}{day}_t{time}z_f02-23.grib2'
      elif time == '14':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_{product}_{year}{month}{day}_t{time}z_f02-21.grib2'
      elif time == '18':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{product}_{year}{month}{day}_t{time}z_f02-17.grib2'
        sigurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_{product}_{year}{month}{day}_t{time}z_f02-17.grib2'
      else:
        raise ValueError('Valid Data For The Date/Time and Hazard Is Not Available For Your Selection')
    elif period == '2':
      if time ==  "12":
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{product}_{year}{month}{day}_t{time}z_f24-47.grib2'
        sigurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_{product}_{year}{month}{day}_t{time}z_f24-47.grib2'
      else:
        raise ValueError('Day 2 Data Is Only Available at 12z')

    r1 = requests.get(url)
    file1 = 'nadocast.grb2'
    with open(file1, 'wb') as f1:
      f1.write(r1.content)

    r2 = requests.get(sigurl)
    file2 = 'nadocast_sig.grb2'
    with open(file2, 'wb') as f2:
      f2.write(r2.content)

  elif  product == 'sig_tornado' or product == 'sig_hail' or product == 'sig_wind':
    if period == '1':
      if time == '00' or time == '0':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_{product}_{year}{month}{day}_t{time}z_f12-35.grib2'
      elif time == '12':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{product}_{year}{month}{day}_t{time}z_f02-23.grib2'
      elif time == '14':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{product}_{year}{month}{day}_t{time}z_f02-21.grib2'
      elif time == '18':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{product}_{year}{month}{day}_t{time}z_f02-17.grib2'
      else:
        raise ValueError('Valid Data For The Date/Time and Hazard Is Not Available For Your Selection')
    elif period == '2':
      if time ==  "12":
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{product}_{year}{month}{day}_t{time}z_f24-47.grib2'
      else:
        raise ValueError('Day 2 Data Is Only Available at 12z')
    r1 = requests.get(url)
    file1 = 'nadocast.grb2'
    with open(file1, 'wb') as f1:
      f1.write(r1.content)

  elif product == 'hail_all' or product == 'wind_all' or product == 'tornado_all':
    if product == 'hail_all':
      product = 'hail'
    elif product == 'wind_all':
      product = 'wind'
    elif product == 'tornado_all':
      product = 'tornado'

    if period == '1':
      if time == '00' or time == '0':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_{product}_{year}{month}{day}_t{time}z_f12-35.grib2'
        sigurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_sig_{product}_{year}{month}{day}_t{time}z_f12-35.grib2'
      elif time == '12':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{product}_{year}{month}{day}_t{time}z_f02-23.grib2'
        sigurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_{product}_{year}{month}{day}_t{time}z_f02-23.grib2'
      elif time == '14':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_{product}_{year}{month}{day}_t{time}z_f02-21.grib2'
      elif time == '18':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{product}_{year}{month}{day}_t{time}z_f02-17.grib2'
        sigurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_{product}_{year}{month}{day}_t{time}z_f02-17.grib2'
      else:
        raise ValueError('Valid Data For The Date/Time and Hazard Is Not Available For Your Selection')
    elif period == '2':
      if time ==  "12":
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{product}_{year}{month}{day}_t{time}z_f24-47.grib2'
        sigurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_{product}_{year}{month}{day}_t{time}z_f24-47.grib2'
      else:
        raise ValueError('Day 2 Data Is Only Available at 12z')

    r1 = requests.get(url)
    file1 = 'nadocast.grb2'
    with open(file1, 'wb') as f1:
      f1.write(r1.content)

    r2 = requests.get(sigurl)
    file2 = 'nadocast_sig.grb2'
    with open(file2, 'wb') as f2:
      f2.write(r2.content)

  elif product == 'all_hazard':
    if time == '00' or time == '0':
        torurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_tornado_{year}{month}{day}_t{time}z_f12-35.grib2'
        sigtorurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_sig_tornado_{year}{month}{day}_t{time}z_f12-35.grib2'
        hailurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_hail_{year}{month}{day}_t{time}z_f12-35.grib2'
        sighailurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_sig_hail_{year}{month}{day}_t{time}z_f12-35.grib2'
        windurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_wind_{year}{month}{day}_t{time}z_f12-35.grib2'
        sigwindurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0/nadocast_2022_models_conus_sig_wind_{year}{month}{day}_t{time}z_f12-35.grib2'

    elif time == '12':
      torurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_tornado_{year}{month}{day}_t{time}z_f02-23.grib2'
      sigtorurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_tornado_{year}{month}{day}_t{time}z_f02-23.grib2'
      hailurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_hail_{year}{month}{day}_t{time}z_f02-23.grib2'
      sighailurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_hail_{year}{month}{day}_t{time}z_f02-23.grib2'
      windurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_wind_{year}{month}{day}_t{time}z_f02-23.grib2'
      sigwindurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}/nadocast_2022_models_conus_sig_wind_{year}{month}{day}_t{time}z_f02-23.grib2'
    elif time == '14':
      torurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_tornado_{year}{month}{day}_t{time}z_f02-21.grib2'
      hailurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_hail_{year}{month}{day}_t{time}z_f02-21.grib2'
      windurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_wind_{year}{month}{day}_t{time}z_f02-21.grib2'
    elif time == '18':
      torurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_tornado_{year}{month}{day}_t{time}z_f02-17.grib2'
      sigtorurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_tornado_{year}{month}{day}_t{time}z_f02-17.grib2'
      hailurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_hail_{year}{month}{day}_t{time}z_f02-17.grib2'
      sighailurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_hail_{year}{month}{day}_t{time}z_f02-17.grib2'
      windurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_wind_{year}{month}{day}_t{time}z_f02-17.grib2'
      sigwindurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}/nadocast_2022_models_conus_sig_wind_{year}{month}{day}_t{time}z_f02-17.grib2'
    else:
      raise ValueError('Valid Data For The Date/Time and Hazard Is Not Available For Your Selection')
  elif period == '2':
    if time ==  "12":
      torurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_tornado_{year}{month}{day}_t{time}z_f24-47.grib2'
      sigtorurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_tornado_{year}{month}{day}_t{time}z_f24-47.grib2'
      hailurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_hail_{year}{month}{day}_t{time}z_f24-47.grib2'
      sighailurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_hail_{year}{month}{day}_t{time}z_f24-47.grib2'
      windurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_wind_{year}{month}{day}_t{time}z_f24-47.grib2'
      sigwindurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}/nadocast_2022_models_conus_sig_wind_{year}{month}{day}_t{time}z_f24-47.grib2'
    else:
      raise ValueError('Day 2 Data Is Only Available at 12z')

  r1 = requests.get(torurl)
  file1 = 'tornado.grb2'
  with open(file1, 'wb') as f1:
    f1.write(r1.content)

  r2 = requests.get(sigtorurl)
  file2 = 'tornado_sig.grb2'
  with open(file2, 'wb') as f2:
    f2.write(r2.content)

  r3 = requests.get(hailurl)
  file3 = 'hail.grb2'
  with open(file3, 'wb') as f3:
    f3.write(r3.content)

elif type_spcabs == 'Absolutey Calibrated':
  if product == 'tornado' or product == 'wind' or product == 'hail':
    if period == '1':
      if time == '00' or time == '0':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_{product}_abs_calib_{year}{month}{day}_t{time}z_f12-35.grib2'
        sigurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_sig_{product}_abs_calib_{year}{month}{day}_t{time}z_f12-35.grib2'
      elif time == '12':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{product}_abs_calib_{year}{month}{day}_t{time}z_f02-23.grib2'
        sigurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_{product}_abs_calib_{year}{month}{day}_t{time}z_f02-23.grib2'
      elif time == '14':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_{product}_abs_calib_{year}{month}{day}_t{time}z_f02-21.grib2'
      elif time == '18':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{product}_abs_calib_{year}{month}{day}_t{time}z_f02-17.grib2'
        sigurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_{product}_abs_calib_{year}{month}{day}_t{time}z_f02-17.grib2'
      else:
        raise ValueError('Valid Data For The Date/Time and Hazard Is Not Available For Your Selection')
    elif period == '2':
      if time ==  "12":
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{product}_abs_calib_{year}{month}{day}_t{time}z_f24-47.grib2'
        sigurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_{product}_abs_calib_{year}{month}{day}_t{time}z_f24-47.grib2'
      else:
        raise ValueError('Day 2 Data Is Only Available at 12z')

    r1 = requests.get(url)
    file1 = 'nadocast.grb2'
    with open(file1, 'wb') as f1:
      f1.write(r1.content)

    r2 = requests.get(sigurl)
    file2 = 'nadocast_sig.grb2'
    with open(file2, 'wb') as f2:
      f2.write(r2.content)

  elif  product == 'sig_tornado' or product == 'sig_hail' or product == 'sig_wind':
    if period == '1':
      if time == '00' or time == '0':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_{product}_abs_calib_{year}{month}{day}_t{time}z_f12-35.grib2'
      elif time == '12':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{product}_abs_calib_{year}{month}{day}_t{time}z_f02-23.grib2'
      elif time == '14':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{product}_abs_calib_{year}{month}{day}_t{time}z_f02-21.grib2'
      elif time == '18':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{product}_abs_calib_{year}{month}{day}_t{time}z_f02-17.grib2'
      else:
        raise ValueError('Valid Data For The Date/Time and Hazard Is Not Available For Your Selection')
    elif period == '2':
      if time ==  "12":
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{product}_abs_calib_{year}{month}{day}_t{time}z_f24-47.grib2'
      else:
        raise ValueError('Day 2 Data Is Only Available at 12z')
    r1 = requests.get(url)
    file1 = 'nadocast.grb2'
    with open(file1, 'wb') as f1:
      f1.write(r1.content)

  elif product == 'hail_all' or product == 'wind_all' or product == 'tornado_all':
    if product == 'hail_all':
      product = 'hail'
    elif product == 'wind_all':
      product = 'wind'
    elif product == 'tornado_all':
      product = 'tornado'

    if period == '1':
      if time == '00' or time == '0':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_{product}_abs_calib_{year}{month}{day}_t{time}z_f12-35.grib2'
        sigurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_sig_{product}_abs_calib_{year}{month}{day}_t{time}z_f12-35.grib2'
      elif time == '12':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{product}_abs_calib_{year}{month}{day}_t{time}z_f02-23.grib2'
        sigurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_{product}_abs_calib_{year}{month}{day}_t{time}z_f02-23.grib2'
      elif time == '14':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_{product}_abs_calib_{year}{month}{day}_t{time}z_f02-21.grib2'
      elif time == '18':
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{product}_abs_calib_{year}{month}{day}_t{time}z_f02-17.grib2'
        sigurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_{product}_abs_calib_{year}{month}{day}_t{time}z_f02-17.grib2'
      else:
        raise ValueError('Valid Data For The Date/Time and Hazard Is Not Available For Your Selection')
    elif period == '2':
      if time ==  "12":
        url = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_{product}_abs_calib_{year}{month}{day}_t{time}z_f24-47.grib2'
        sigurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_{product}_abs_calib_{year}{month}{day}_t{time}z_f24-47.grib2'
      else:
        raise ValueError('Day 2 Data Is Only Available at 12z')

    r1 = requests.get(url)
    file1 = 'nadocast.grb2'
    with open(file1, 'wb') as f1:
      f1.write(r1.content)

    r2 = requests.get(sigurl)
    file2 = 'nadocast_sig.grb2'
    with open(file2, 'wb') as f2:
      f2.write(r2.content)

  elif product == 'all_hazard':
    if period == '1':
      if time == '00' or time == '0':
          torurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_tornado_abs_calib_{year}{month}{day}_t{time}z_f12-35.grib2'
          sigtorurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_sig_tornado_abs_calib_{year}{month}{day}_t{time}z_f12-35.grib2'
          hailurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_hail_abs_calib_{year}{month}{day}_t{time}z_f12-35.grib2'
          sighailurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_sig_hail_abs_calib_{year}{month}{day}_t{time}z_f12-35.grib2'
          windurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0z/nadocast_2022_models_conus_wind_abs_calib_{year}{month}{day}_t{time}z_f12-35.grib2'
          sigwindurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t0/nadocast_2022_models_conus_sig_wind_abs_calib_{year}{month}{day}_t{time}z_f12-35.grib2'

      elif time == '12':
        torurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_tornado_abs_calib_{year}{month}{day}_t{time}z_f02-23.grib2'
        sigtorurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_tornado_abs_calib_{year}{month}{day}_t{time}z_f02-23.grib2'
        hailurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_hail_abs_calib_{year}{month}{day}_t{time}z_f02-23.grib2'
        sighailurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_hail_abs_calib_{year}{month}{day}_t{time}z_f02-23.grib2'
        windurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_wind_abs_calib_{year}{month}{day}_t{time}z_f02-23.grib2'
        sigwindurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}/nadocast_2022_models_conus_sig_wind_abs_calib_{year}{month}{day}_t{time}z_f02-23.grib2'
      elif time == '14':
        torurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_tornado_abs_calib_{year}{month}{day}_t{time}z_f02-21.grib2'
        hailurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_hail_abs_calib_{year}{month}{day}_t{time}z_f02-21.grib2'
        windurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_wind_abs_calib_{year}{month}{day}_t{time}z_f02-21.grib2'
      elif time == '18':
        torurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_tornado_abs_calib_{year}{month}{day}_t{time}z_f02-17.grib2'
        sigtorurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_tornado_abs_calib_{year}{month}{day}_t{time}z_f02-17.grib2'
        hailurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_hail_abs_calib_{year}{month}{day}_t{time}z_f02-17.grib2'
        sighailurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_hail_abs_calib_{year}{month}{day}_t{time}z_f02-17.grib2'
        windurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_wind_abs_calib_{year}{month}{day}_t{time}z_f02-17.grib2'
        sigwindurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}/nadocast_2022_models_conus_sig_wind_abs_calib_{year}{month}{day}_t{time}z_f02-17.grib2'
      else:
        raise ValueError('Valid Data For The Date/Time and Hazard Is Not Available For Your Selection')
    elif period == '2':
      if time ==  "12":
        torurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_tornado_abs_calib_{year}{month}{day}_t{time}z_f24-47.grib2'
        sigtorurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_tornado_abs_calib_{year}{month}{day}_t{time}z_f24-47.grib2'
        hailurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_hail_abs_calib_{year}{month}{day}_t{time}z_f24-47.grib2'
        sighailurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_sig_hail_abs_calib_{year}{month}{day}_t{time}z_f24-47.grib2'
        windurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}z/nadocast_2022_models_conus_wind_abs_calib_{year}{month}{day}_t{time}z_f24-47.grib2'
        sigwindurl = f'http://data.nadocast.com/{year}{month}/{year}{month}{day}/t{time}/nadocast_2022_models_conus_sig_wind_abs_calib_{year}{month}{day}_t{time}z_f24-47.grib2'
      else:
        raise ValueError('Day 2 Data Is Only Available at 12z')

    r1 = requests.get(torurl)
    file1 = 'tornado.grb2'
    with open(file1, 'wb') as f1:
      f1.write(r1.content)

    r2 = requests.get(sigtorurl)
    file2 = 'tornado_sig.grb2'
    with open(file2, 'wb') as f2:
      f2.write(r2.content)

    r3 = requests.get(hailurl)
    file3 = 'hail.grb2'
    with open(file3, 'wb') as f3:
      f3.write(r3.content)

else:
  raise ValueError('Invalid Product Selected')

#Open Nadocast File

In [ ]:
# @title
%%capture
nadocastfile = xr.open_mfdataset('/content/nadocast.grb2', engine="cfgrib")

if product == 'tornado' or product == 'wind' or product == 'hail':
  signadocastfile = xr.open_mfdataset('/content/nadocast_sig.grb2', engine="cfgrib")

if product == 'tornado':
  prob = nadocastfile.torprob
  product_label = "Tornado"
  sigprob = signadocastfile.storprob

  levels = [0.1,1,2,3,5,10,15,30,45,60, 100]
  color_list = ['#f4f4f4','#dcdcdc','#118a14','#53cf55','#8a472a','#fec72e','#fc0d1b','#fc28fc','#9039ea','#154f89']

  siglevel = [10,100]
  sig_hatch = ['////']
  sig_color = ['none']

  PN1 = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#f4f4f4')
  ONE = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#dcdcdc')
  TWO = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#118a14')
  THR = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#53cf55')
  FIV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#8a472a')
  TEN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fec72e')
  FTN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc0d1b')
  TTY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc28fc')
  FFV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#9039ea')
  SXY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#154f89')
  SIG = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', hatch = '////', facecolor = 'none')

  legend_patches = [PN1, ONE, TWO, THR, FIV, TEN, FTN, TTY, FFV, SXY, SIG]
  legend_labels = ['0.1%','1%','2%','3%','5%','10%','15%','30%','45%','60%', 'SIG']

elif product == 'hail':
  prob = nadocastfile.hailprob
  product_label = "Hail"
  sigprob = signadocastfile.shailpro

  levels = [1,5,15,30,45,60, 100]
  color_list = ['#e6e6e6', '#8a472a', '#fec72e', '#fc0d1b', '#fc28fc', '#9039ea']
  siglevel = [10,100]
  sig_hatch = ['////']
  sig_color = ['none']

  ONE = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#e6e6e6')
  FIV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#8a472a')
  FTN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fec72e')
  TTY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc0d1b')
  FFV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc28fc')
  SXY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#9039ea')
  SIG = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', hatch = '////', facecolor = 'none')

  legend_patches = [ONE, FIV, FTN, TTY, FFV, SXY, SIG]
  legend_labels = ['1%','5%','15%','30%','45%','60%', 'SIG']


elif product == 'wind':
  prob = nadocastfile.windprob
  product_label = "Wind"
  sigprob = signadocastfile.swindpro

  levels = [1,5,15,30,45,60, 100]
  color_list = ['#e6e6e6', '#8a472a', '#fec72e', '#fc0d1b', '#fc28fc', '#9039ea']
  siglevel = [10,100]
  sig_hatch = ['////']
  sig_color = ['none']

  ONE = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#e6e6e6')
  FIV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#8a472a')
  FTN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fec72e')
  TTY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc0d1b')
  FFV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc28fc')
  SXY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#9039ea')
  SIG = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', hatch = '////', facecolor = 'none')

  legend_patches = [ONE, FIV, FTN, TTY, FFV, SXY, SIG]
  legend_labels = ['1%','5%','15%','30%','45%','60%','SIG']

elif product == 'sig_tornado':
  prob = nadocastfile.storprob
  product_label = "Significant Tornado"

  levels = [0.1,1,2,3,5,10,15,30,45,60, 100]
  color_list = ['#f4f4f4','#dcdcdc','#118a14','#53cf55','#8a472a','#4d4d4d','#fc0d1b','#fc28fc','#9039ea','#154f89']

  PN1 = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#f4f4f4')
  ONE = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#dcdcdc')
  TWO = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#118a14')
  THR = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#53cf55')
  FIV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#8a472a')
  TEN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#4d4d4d')
  FTN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc0d1b')
  TTY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc28fc')
  FFV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#9039ea')
  SXY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#154f89')
  SIG = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', hatch = '////', facecolor = 'none')

  legend_patches = [PN1, ONE, TWO, THR, FIV, TEN, FTN, TTY, FFV, SXY, SIG]
  legend_labels = ['0.1%','1%','2%','3%','5%','10%','15%','30%','45%','60%', 'SIG']



elif product == 'sig_hail':
  prob = nadocastfile.shailpro
  product_label = "Significant Hail"

  levels = [1,5,10,15,30,45,60, 100]
  color_list = ['#e6e6e6', '#8a472a', '#4d4d4d', '#fec72e', '#fc0d1b', '#fc28fc', '#9039ea']

  ONE = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#e6e6e6')
  FIV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#8a472a')
  TEN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#4d4d4d')
  FTN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fec72e')
  TTY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc0d1b')
  FFV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc28fc')
  SXY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#9039ea')

  legend_patches = [ONE, FIV, TEN, FTN, TTY, FFV, SXY]
  legend_labels = ['1%','5%','10%', '15%','30%','45%','60%']



elif product == 'sig_wind':
  prob = nadocastfile.swindpro
  product_label = "Significant Wind"

  levels = [1,5,10,15,30,45,60, 100]
  color_list = ['#e6e6e6', '#8a472a', '#4d4d4d', '#fec72e', '#fc0d1b', '#fc28fc', '#9039ea']


  ONE = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#e6e6e6')
  FIV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#8a472a')
  TEN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#4d4d4d')
  FTN = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fec72e')
  TTY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc0d1b')
  FFV = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#fc28fc')
  SXY = mpatches.Rectangle((0, 0), 1, 0.1, linewidth=2, edgecolor='black', facecolor='#9039ea')

  legend_patches = [ONE, FIV, TEN, FTN, TTY, FFV, SXY]
  legend_labels = ['1%','5%','10%', '15%','30%','45%','60%']



In [ ]:
# @title
#Set Up Figure
fig = plt.figure(figsize=(12, 12), facecolor='white', edgecolor="black", linewidth =6)
ax=fig.add_subplot(1,1,1, projection=ccrs.PlateCarree())
ax.set_extent((WestLong, EastLong, SouthLat, NorthLat))
ax.set_box_aspect(0.7)
box1 = ax.get_position()
ax.set_position([box1.x0, box1.y0, box1.width * 0.8, box1.height*0.95])

#Plot Data
nado_contour = ax.contourf(prob.longitude, prob.latitude, prob.values, levels = levels, colors = color_list)
if product == 'tornado' or product == 'hail' or product == 'wind':
  sig_contour = ax.contourf(sigprob.longitude, sigprob.latitude, sigprob.values, levels = siglevel, colors = sig_color, hatches = sig_hatch)
  sig_line = ax.contour(sigprob.longitude, sigprob.latitude, sigprob.values, levels = siglevel, colors = 'black', linewidths = 1)

#Create Legend
ax.legend(legend_patches, legend_labels,  loc='upper left', fontsize=12, facecolor='white', markerscale=12,
              labelcolor='k', title='  Risk', title_fontsize=12, edgecolor='black', framealpha=1,bbox_to_anchor=(1.10, 1), fancybox=True, shadow=True,)

#Add Counties and Coastlines
ax.coastlines(color='black',resolution='10m',zorder=99)
ax.add_feature(cfeat.STATES.with_scale('50m'),edgecolor='black', zorder=9, linewidth = 4)
ax.add_feature(USCOUNTIES, facecolor='None', edgecolor='darkgray', zorder=9)
lake = cfeat.NaturalEarthFeature('physical', 'lakes', '50m', edgecolor='blue', facecolor='blue', alpha=0.2)
ax.add_feature(lake)

#Add Figure Text
plt.figtext(0.11, 0.23, f'SCOTT THOMAS | SOURCE:Nadocast', weight='bold', color='black', fontsize=13, ha='left', alpha=0.8)

if type_spcabs == 'SPC Like':
  if period == '1':
    plt.figtext(0.45, 0.73, f'Nadocast Day 1 {product_label} Risk {month}-{day}-{year} {time} UTC Run', ha='center', weight='bold', fontsize=16)
  if period == '2':
    plt.figtext(0.45, 0.73, f'Nadocast Day 2 {product_label} Risk {month}-{day}-{year} {time} UTC Run', ha='center', weight='bold', fontsize=16)
elif type_spcabs == 'Absolutey Calibrated':
  if period == '1':
    plt.figtext(0.45, 0.73, f'Nadocast Day 1 Absolutely Calibrated {product_label} Risk {month}-{day}-{year} {time} UTC Run', ha='center', weight='bold', fontsize=16)
  if period == '2':
    plt.figtext(0.45, 0.73, f'Nadocast Day 2 Absolutely Calibrated {product_label} Risk {month}-{day}-{year} {time} UTC Run', ha='center', weight='bold', fontsize=16)

#Add Gridlines
gl = ax.gridlines(draw_labels = True, x_inline = False, y_inline = False, rotate_labels = False, linewidth = 0.5, linestyle = '--', color = 'black', alpha = 0.1)
gl.x_labels_top = False
gl.x_labels_bottom = True

#Save Image
if type_spcabs == 'SPC Like':
  plt.savefig(f'Nadocast_Day{period}_{product}_{year}-{month}-{day}-{time}UTC.png', bbox_inches = 'tight')
elif type_spcabs == 'Absolutey Calibrated':
  plt.savefig(f'Nadocast_Day{period}_{product}_Abs_Calib_{year}-{month}-{day}-{time}UTC.png', bbox_inches = 'tight')